In [ ]:
import os
import json
import re
import jieba

from opencc import OpenCC
from os import listdir
from gensim.models import word2vec

def sc_to_tc(string): #簡體中文轉繁體中文
    cc = OpenCC('s2t') 
    return cc.convert(string)

def remain_language(string): #留下文字
    result = re.sub('\W+', ' ', string).replace("_",  '')
    return result

def transform_and_write_data(file_path): #把wiki_zh濃縮成一個.txt
    files_layer_1 = listdir(file_path)
    #print(files_layer_1)
    
    with open( "./wiki_zh_seq.txt" , 'w', encoding='utf-8') as new_file: 
        for file_layer_1 in files_layer_1:
            if file_layer_1=='.DS_Store':
                continue
            else:
                files_layer_2 = listdir(file_path+"/"+file_layer_1)
                #print(files_layer_2)
                
                for file_layer_2 in files_layer_2:
                    with open( file_path+"/"+file_layer_1+"/"+file_layer_2 , 'r', encoding='utf-8') as file: #開wiki_zh的每個txt檔
                        for times, data in enumerate(file, 1):
                            print('data num:', times)
                            data = json.loads(data)
                            data = data['text']
                            data =remain_language(data)
                            data = sc_to_tc(data) #轉繁體中文
                            data = jieba.lcut(data) 
                            data = [word for word in data if word != ' ']
                            data = ' '.join(data)+'\n'

                            new_file.write(data)


In [ ]:
def training(train_data_path): #訓練並儲存模型
    
    seed = 666 #亂數種子
    sg = 0 #用CBOW(0)或是Skip-gram(1)
    window_size = 10 #周圍詞彙要看多少範圍
    vector_size = 100 #轉成向量的維度
    min_count = 1 #詞頻少於 min_count 之詞彙不會參與訓練
    workers = 8 #訓練的並行數量
    epochs = 5 #訓練的迭代次數
    batch_words = 10000 #每次給予多少詞彙量訓練

    train_data = word2vec.LineSentence(train_data_path)
    
    model = word2vec.Word2Vec(
        train_data,
        min_count=min_count,
        vector_size=vector_size,
        workers=workers,
        epochs=epochs,
        window=window_size,
        sg=sg,
        seed=seed,
        batch_words=batch_words
    )

    model.save('word2vec.model') #儲存模型


In [ ]:
if __name__=="__main__":
    file_path="./wiki_zh"
    train_data_path="./wiki_zh_seq.txt"
    
    #transform_and_write_data(file_path)
    
    training(train_data_path) #訓練模型
    
    model = word2vec.Word2Vec.load('word2vec.model')

    for item in model.wv.most_similar('生物', topn=20):
        print(item)
    